In [30]:
import os
from typing import List, Dict, Any, Union
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool
from langchain.callbacks import get_openai_callback
from pydantic import BaseModel, Field
from langchain.agents.agent import AgentOutputParser
import re
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ConversationBufferMemory

# Load environment variables
from dotenv import load_dotenv
load_dotenv()


False

In [ ]:

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Initialize OpenAI LLM
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [9]:
# Define prompt templates
city_ranking_template = PromptTemplate(
    input_variables=["country", "budget", "needs"],
    template="Rank the top 5 cities in {country} for real estate investment with a budget of {budget} and these needs: {needs}. Provide a brief explanation for each city."
)

neighborhood_analysis_template = PromptTemplate(
    input_variables=["city", "budget", "needs"],
    template="Analyze the top 5 neighborhoods in {city} for real estate investment with a budget of {budget} and these needs: {needs}. Include pros and cons for each neighborhood."
)

# Define prompt templates
city_ranking_template = PromptTemplate(
    input_variables=["country", "budget", "needs"],
    template="Rank the top 5 cities in {country} for real estate investment with a budget of {budget} and these needs: {needs}. Provide a brief explanation for each city."
)

neighborhood_analysis_template = PromptTemplate(
    input_variables=["city", "budget", "needs"],
    template="Analyze the top 5 neighborhoods in {city} for real estate investment with a budget of {budget} and these needs: {needs}. Include pros and cons for each neighborhood."
)

property_recommendation_template = PromptTemplate(
    input_variables=["neighborhood", "budget", "needs"],
    template="Recommend 3 types of properties in {neighborhood} that fit a budget of {budget} and these needs: {needs}. Include estimated prices and potential return on investment."
)

market_trend_template = PromptTemplate(
    input_variables=["location"],
    template="Analyze the real estate market trends in {location} over the past 5 years and provide a forecast for the next 2 years."
)

In [10]:
# Create LLMChains
city_ranking_chain = LLMChain(llm=llm, prompt=city_ranking_template)
neighborhood_analysis_chain = LLMChain(llm=llm, prompt=neighborhood_analysis_template)
property_recommendation_chain = LLMChain(llm=llm, prompt=property_recommendation_template)
market_trend_chain = LLMChain(llm=llm, prompt=market_trend_template)


In [11]:
class CityRankingTool(BaseTool):
    name = "CityRanking"
    description = "Useful for ranking cities based on real estate criteria"

    def _run(self, country: str, budget: str, needs: str) -> str:
        return city_ranking_chain.run(country=country, budget=budget, needs=needs)

    def _arun(self, country: str, budget: str, needs: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [12]:
class NeighborhoodAnalysisTool(BaseTool):
    name = "NeighborhoodAnalysis"
    description = "Useful for analyzing neighborhoods within a city"

    def _run(self, city: str, budget: str, needs: str) -> str:
        return neighborhood_analysis_chain.run(city=city, budget=budget, needs=needs)

    def _arun(self, city: str, budget: str, needs: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [13]:
class PropertyRecommendationTool(BaseTool):
    name = "PropertyRecommendation"
    description = "Useful for recommending properties in a specific neighborhood"

    def _run(self, neighborhood: str, budget: str, needs: str) -> str:
        return property_recommendation_chain.run(neighborhood=neighborhood, budget=budget, needs=needs)

    def _arun(self, neighborhood: str, budget: str, needs: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [14]:
class MarketTrendTool(BaseTool):
    name = "MarketTrend"
    description = "Useful for analyzing real estate market trends in a location"

    def _run(self, location: str) -> str:
        return market_trend_chain.run(location=location)

    def _arun(self, location: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [15]:
# Define tools
tools = [
    CityRankingTool(),
    NeighborhoodAnalysisTool(),
    PropertyRecommendationTool(),
    MarketTrendTool()
]

In [39]:
# Define a custom output parser
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if the output indicates a final answer
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Use regex to parse the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        
        action = match.group(1).strip()
        action_input = match.group(2)
        
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

# Update the agent 

In [40]:
agent = LLMSingleActionAgent(
    llm_chain=LLMChain(llm=llm, prompt=PromptTemplate(template="{input}\nThought: {agent_scratchpad}", input_variables=["input", "agent_scratchpad"])),
    output_parser=CustomOutputParser(),
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

In [41]:
import json
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ConversationBufferMemory

# ... (keep all the previous code up to the tool definitions)

# Wrapper functions for tools
def city_ranking_wrapper(input_str):
    try:
        input_dict = json.loads(input_str)
        return city_ranking_chain.run(**input_dict)
    except json.JSONDecodeError:
        return "Error: Input should be a JSON string with keys 'country', 'budget', and 'needs'."

def neighborhood_analysis_wrapper(input_str):
    try:
        input_dict = json.loads(input_str)
        return neighborhood_analysis_chain.run(**input_dict)
    except json.JSONDecodeError:
        return "Error: Input should be a JSON string with keys 'city', 'budget', and 'needs'."

def property_recommendation_wrapper(input_str):
    try:
        input_dict = json.loads(input_str)
        return property_recommendation_chain.run(**input_dict)
    except json.JSONDecodeError:
        return "Error: Input should be a JSON string with keys 'neighborhood', 'budget', and 'needs'."

def market_trend_wrapper(input_str):
    try:
        input_dict = json.loads(input_str)
        return market_trend_chain.run(**input_dict)
    except json.JSONDecodeError:
        return "Error: Input should be a JSON string with key 'location'."

# Define tools with wrappers
tools = [
    Tool(
        name="CityRanking",
        func=city_ranking_wrapper,
        description="Useful for ranking cities based on real estate criteria. Input should be a JSON string with keys 'country', 'budget', and 'needs'."
    ),
    Tool(
        name="NeighborhoodAnalysis",
        func=neighborhood_analysis_wrapper,
        description="Useful for analyzing neighborhoods within a city. Input should be a JSON string with keys 'city', 'budget', and 'needs'."
    ),
    Tool(
        name="PropertyRecommendation",
        func=property_recommendation_wrapper,
        description="Useful for recommending properties in a specific neighborhood. Input should be a JSON string with keys 'neighborhood', 'budget', and 'needs'."
    ),
    Tool(
        name="MarketTrend",
        func=market_trend_wrapper,
        description="Useful for analyzing real estate market trends in a location. Input should be a JSON string with key 'location'."
    )
]

# Custom prompt template (INSERT HERE)
template = """You are an AI real estate agent. Your task is to provide real estate investment recommendations in a specific JSON format.

Your recommendations should include:
1. One country
2. Up to three cities in that country
3. Up to ten neighborhoods for each city

Use the following tools to gather information:
{tools}
"""


In [42]:
# Main function to run the AI real estate agent
def run_ai_real_estate_agent():
    print("Welcome to the AI Real Estate Agent!")
    print("I can help you with real estate investment decisions.")
    print("Please provide the following information:")

    country = input("Country: ")
    budget = input("Budget: ")
    needs = input("Your needs (comma-separated): ")

    input_data = {
        "country": country,
        "budget": budget,
        "needs": needs
    }

    try:
        with get_openai_callback() as cb:
            result = agent_executor.run(input_data)
            print("\nAI Real Estate Agent Report:")
            print(result)
            print(f"\nTotal Tokens: {cb.total_tokens}")
            print(f"Total Cost (USD): ${cb.total_cost:.4f}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    run_ai_real_estate_agent()

Welcome to the AI Real Estate Agent!
I can help you with real estate investment decisions.
Please provide the following information:


Country:  Portugal
Budget:  200000
Your needs (comma-separated):  apartments between T0 to T3 in a good location for investment 




> Entering new AgentExecutor chain...
An error occurred: Missing some input keys: {'input', 'agent_scratchpad'}
